In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
strategy = tf.distribute.MirroredStrategy()

2024-11-01 12:21:27.965695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-01 12:21:27.987918: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-01 12:21:27.987957: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-01 12:21:28.001739: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-01 12:21:29.169439: W tensorflow/compiler/tf

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2024-11-01 12:21:33.801082: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38380 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:87:00.0, compute capability: 8.0
2024-11-01 12:21:33.806858: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38380 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:bd:00.0, compute capability: 8.0


In [ ]:
from data_loader.create_dataset import create_tf_datasets

from models.attention_unet import AttentionUNet3D
from models.unet import UNet3D

from trainer.train import train_model

# BC Dataset

In [3]:
train_dir = 'data/BC/training_data/'

train_dataset, val_dataset = create_tf_datasets(
    train_dir, 
    percent_val=0.2, 
    patch_shape=(64, 64, 64), 
    patch_step=64,
    random_state = 42
)

Processing img1 image and mask
Processing img2 image and mask


In [4]:
with strategy.scope():
    train_model(
        train_dataset = train_dataset, 
        val_dataset = val_dataset, 
        model = UNet3D().build_model(), 
        optimizer = 'adam', 
        loss = 'binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall'],
        epochs = 50,
        batch_size = 16, 
        filename="bc_unet3d.keras",
        model_name="bc_unet3d"
    )

Epoch 1/50


2024-10-31 19:11:02.696320: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2024-10-31 19:11:02.699355: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


67/68 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step - accuracy: 0.9503 - loss: 124.2837 - precision: 0.0770 - recall: 0.0234

2024-10-31 19:11:30.319094: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-10-31 19:11:30.319225: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-10-31 19:11:30.319272: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-10-31 19:11:30.319360: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step - accuracy: 0.9505 - loss: 123.0096 - precision: 0.0801 - recall: 0.0242

...

68/68 ━━━━━━━━━━━━━━━━━━━━ 31s 456ms/step - accuracy: 0.9923 - loss: 0.0179 - precision: 0.9034 - recall: 0.8876 - val_accuracy: 0.9918 - val_loss: 0.0193 - val_precision: 0.8756 - val_recall: 0.9245
Epoch 49/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - accuracy: 0.9922 - loss: 0.0183 - precision: 0.8954 - recall: 0.8899

2024-10-31 19:38:38.807315: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-10-31 19:38:38.807417: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-10-31 19:38:38.807467: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-10-31 19:38:38.807515: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-10-31 19:38:40.549284: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequen

68/68 ━━━━━━━━━━━━━━━━━━━━ 30s 426ms/step - accuracy: 0.9922 - loss: 0.0183 - precision: 0.8956 - recall: 0.8900 - val_accuracy: 0.9921 - val_loss: 0.0182 - val_precision: 0.9210 - val_recall: 0.8750
Epoch 50/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step - accuracy: 0.9923 - loss: 0.0181 - precision: 0.8971 - recall: 0.8944

2024-10-31 19:39:07.671876: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-10-31 19:39:07.671988: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-10-31 19:39:07.672145: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-10-31 19:39:07.672227: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-10-31 19:39:09.476278: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequen

68/68 ━━━━━━━━━━━━━━━━━━━━ 29s 419ms/step - accuracy: 0.9923 - loss: 0.0180 - precision: 0.8973 - recall: 0.8945 - val_accuracy: 0.9921 - val_loss: 0.0185 - val_precision: 0.9254 - val_recall: 0.8690


In [5]:
with strategy.scope():
    train_model(
        train_dataset = train_dataset, 
        val_dataset = val_dataset, 
        model = AttentionUNet3D().build_model(), 
        optimizer = 'adam',
        loss = 'binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall'],
        epochs = 50,
        batch_size = 16,
        filename="bc_attentionunet3d.keras",
        model_name="bc_attentionunet3d"
    )

Epoch 1/50
67/68 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step - accuracy: 0.9264 - loss: 0.1242 - precision: 0.4492 - recall: 0.7547

2024-10-31 19:40:54.130715: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-10-31 19:40:54.130835: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-10-31 19:40:54.132520: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-10-31 19:40:54.132576: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step - accuracy: 0.9271 - loss: 0.1231 - precision: 0.4521 - recall: 0.7555

2024-10-31 19:41:00.277404: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-10-31 19:41:00.277533: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-10-31 19:41:00.278158: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-10-31 19:41:00.278622: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


68/68 ━━━━━━━━━━━━━━━━━━━━ 113s 768ms/step - accuracy: 0.9278 - loss: 0.1220 - precision: 0.4548 - recall: 0.7563 - val_accuracy: 0.9895 - val_loss: 0.0256 - val_precision: 0.8566 - val_recall: 0.8802
Epoch 2/50
67/68 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step - accuracy: 0.9907 - loss: 0.0228 - precision: 0.8789 - recall: 0.8709

...

68/68 ━━━━━━━━━━━━━━━━━━━━ 44s 639ms/step - accuracy: 0.9929 - loss: 0.0165 - precision: 0.9036 - recall: 0.9048 - val_accuracy: 0.9923 - val_loss: 0.0178 - val_precision: 0.9217 - val_recall: 0.8809
Epoch 50/50
67/68 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step - accuracy: 0.9929 - loss: 0.0164 - precision: 0.9097 - recall: 0.9031

2024-10-31 20:12:03.284981: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-10-31 20:12:03.285283: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-10-31 20:12:03.285701: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-10-31 20:12:03.285816: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step - accuracy: 0.9929 - loss: 0.0164 - precision: 0.9096 - recall: 0.9031

2024-10-31 20:12:05.643983: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-10-31 20:12:05.644118: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-10-31 20:12:05.644186: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-10-31 20:12:05.644259: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


68/68 ━━━━━━━━━━━━━━━━━━━━ 34s 500ms/step - accuracy: 0.9929 - loss: 0.0164 - precision: 0.9096 - recall: 0.9031 - val_accuracy: 0.9920 - val_loss: 0.0188 - val_precision: 0.9031 - val_recall: 0.8936


# BC Augmented Dataset

In [3]:
train_dir = 'data/BC/training_data_augm/'

train_dataset, val_dataset = create_tf_datasets(
    train_dir, 
    percent_val=0.2, 
    patch_shape=(64, 64, 64), 
    patch_step=64,
    random_state = 42
)

Processing img1 image and mask
Processing img1_10_SNR image and mask
Processing img1_15_SNR image and mask
Processing img1_1_SNR image and mask
Processing img1_2_SNR image and mask
Processing img1_3_SNR image and mask
Processing img1_4_SNR image and mask
Processing img1_5_SNR image and mask
Processing img2 image and mask
Processing img2_10_SNR image and mask
Processing img2_15_SNR image and mask
Processing img2_1_SNR image and mask
Processing img2_2_SNR image and mask
Processing img2_3_SNR image and mask
Processing img2_4_SNR image and mask
Processing img2_5_SNR image and mask


In [4]:
with strategy.scope():
    train_model(
        train_dataset = train_dataset, 
        val_dataset = val_dataset, 
        model = UNet3D().build_model(), 
        optimizer = 'adam', 
        loss = 'binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall'],
        epochs = 50,
        batch_size = 16, 
        filename="bc_augmented_unet3d.keras",
        model_name="bc_augmented_unet3d"
    )

2024-10-31 23:53:58.989114: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 9079619584 exceeds 10% of free system memory.
2024-10-31 23:54:07.645788: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 9079619584 exceeds 10% of free system memory.
2024-10-31 23:54:11.978330: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 9079619584 exceeds 10% of free system memory.
2024-10-31 23:54:15.339329: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 9079619584 exceeds 10% of free system memory.
2024-10-31 23:54:22.990139: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 9079619584 exceeds 10% of free system memory.


Epoch 1/50


2024-10-31 23:54:44.819746: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2024-10-31 23:54:44.820002: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


541/542 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step - accuracy: 0.9654 - loss: 0.3059 - precision: 0.5129 - recall: 0.2168

2024-10-31 23:57:17.044669: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-10-31 23:57:17.044893: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-10-31 23:57:17.044957: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-10-31 23:57:17.044985: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


...

542/542 ━━━━━━━━━━━━━━━━━━━━ 156s 287ms/step - accuracy: 0.9777 - loss: 0.0883 - precision: 0.4571 - recall: 0.4490 - val_accuracy: 0.9928 - val_loss: 0.0169 - val_precision: 0.9082 - val_recall: 0.8978
Epoch 41/50
542/542 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step - accuracy: 0.9778 - loss: 0.0880 - precision: 0.4566 - recall: 0.4488

2024-11-01 01:44:08.557063: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 01:44:08.557355: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 01:44:08.557777: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 01:44:08.557843: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 01:44:20.608620: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequen

542/542 ━━━━━━━━━━━━━━━━━━━━ 155s 286ms/step - accuracy: 0.9778 - loss: 0.0878 - precision: 0.4575 - recall: 0.4497 - val_accuracy: 0.9924 - val_loss: 0.0179 - val_precision: 0.8791 - val_recall: 0.9249
Epoch 42/50
542/542 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step - accuracy: 0.9777 - loss: 0.0882 - precision: 0.4550 - recall: 0.4462

2024-11-01 01:46:43.736748: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 01:46:43.737036: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 01:46:43.737119: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 01:46:43.737153: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 01:46:56.581884: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequen

542/542 ━━━━━━━━━━━━━━━━━━━━ 156s 287ms/step - accuracy: 0.9778 - loss: 0.0881 - precision: 0.4558 - recall: 0.4470 - val_accuracy: 0.9928 - val_loss: 0.0168 - val_precision: 0.9079 - val_recall: 0.8994


In [5]:
with strategy.scope():
    train_model(
        train_dataset = train_dataset, 
        val_dataset = val_dataset, 
        model = AttentionUNet3D().build_model(), 
        optimizer = 'adam',
        loss = 'binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall'],
        epochs = 50,
        batch_size = 16,
        filename="bc_augmented_attentionunet3d.keras",
        model_name="bc_augmented_attentionunet3d"
    )

Epoch 1/50
541/542 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step - accuracy: 0.9819 - loss: 0.0450 - precision: 0.7701 - recall: 0.7731

2024-11-01 01:51:16.029719: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 01:51:16.030039: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 01:51:16.030162: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 01:51:16.030396: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


542/542 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step - accuracy: 0.9819 - loss: 0.0450 - precision: 0.7702 - recall: 0.7732

2024-11-01 01:51:43.244362: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 01:51:43.244440: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 01:51:43.244535: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 01:51:43.244582: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


542/542 ━━━━━━━━━━━━━━━━━━━━ 258s 412ms/step - accuracy: 0.9819 - loss: 0.0449 - precision: 0.7704 - recall: 0.7733 - val_accuracy: 0.9905 - val_loss: 0.0231 - val_precision: 0.8854 - val_recall: 0.8568
Epoch 2/50
542/542 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step - accuracy: 0.9909 - loss: 0.0217 - precision: 0.8887 - recall: 0.8678

...

542/542 ━━━━━━━━━━━━━━━━━━━━ 208s 383ms/step - accuracy: 0.9951 - loss: 0.0113 - precision: 0.9362 - recall: 0.9314 - val_accuracy: 0.9939 - val_loss: 0.0150 - val_precision: 0.9186 - val_recall: 0.9191
Epoch 49/50
542/542 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step - accuracy: 0.9951 - loss: 0.0111 - precision: 0.9372 - recall: 0.9321

2024-11-01 04:35:54.277693: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 04:35:54.277920: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 04:35:54.278004: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 04:35:54.278093: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 04:36:09.226297: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequen

542/542 ━━━━━━━━━━━━━━━━━━━━ 201s 370ms/step - accuracy: 0.9951 - loss: 0.0111 - precision: 0.9372 - recall: 0.9321 - val_accuracy: 0.9939 - val_loss: 0.0151 - val_precision: 0.9116 - val_recall: 0.9269
Epoch 50/50
542/542 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step - accuracy: 0.9952 - loss: 0.0110 - precision: 0.9381 - recall: 0.9333

2024-11-01 04:39:15.408960: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 04:39:15.409266: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 04:39:15.409329: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 04:39:15.409377: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 04:39:30.337143: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequen

542/542 ━━━━━━━━━━━━━━━━━━━━ 201s 370ms/step - accuracy: 0.9952 - loss: 0.0110 - precision: 0.9381 - recall: 0.9333 - val_accuracy: 0.9939 - val_loss: 0.0152 - val_precision: 0.9060 - val_recall: 0.9332


# Sinusoid Dataset

In [6]:
train_dir = 'data/Sinusoids/training_data/'

train_dataset, val_dataset = create_tf_datasets(
    train_dir, 
    percent_val=0.2, 
    patch_shape=(64, 64, 64), 
    patch_step=64,
    random_state = 42
)

Processing img1 image and mask
Processing img2 image and mask


In [7]:
with strategy.scope():
    train_model(
        train_dataset = train_dataset, 
        val_dataset = val_dataset, 
        model = UNet3D().build_model(), 
        optimizer = 'adam', 
        loss = 'binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall'],
        epochs = 50,
        batch_size = 16, 
        filename="sinusoids_unet3d.keras",
        model_name="sinusoids_unet3d"
    )

Epoch 1/50
67/68 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step - accuracy: 0.8436 - loss: 858.4443 - precision: 0.7423 - recall: 0.0231

2024-11-01 04:40:16.826498: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 04:40:16.826891: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 04:40:16.826919: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 04:40:16.826978: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step - accuracy: 0.8437 - loss: 849.6271 - precision: 0.7416 - recall: 0.0248

2024-11-01 04:40:23.728647: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 04:40:23.728744: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 04:40:23.728812: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 04:40:23.728882: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


68/68 ━━━━━━━━━━━━━━━━━━━━ 49s 525ms/step - accuracy: 0.8439 - loss: 839.5898 - precision: 0.7416 - recall: 0.0254 - val_accuracy: 0.8953 - val_loss: 0.2332 - val_precision: 0.6263 - val_recall: 0.8158
Epoch 2/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step - accuracy: 0.9111 - loss: 0.2665 - precision: 0.7552 - recall: 0.6448

...

68/68 ━━━━━━━━━━━━━━━━━━━━ 24s 347ms/step - accuracy: 0.9670 - loss: 0.0769 - precision: 0.8990 - recall: 0.8901 - val_accuracy: 0.9667 - val_loss: 0.0789 - val_precision: 0.8809 - val_recall: 0.9093
Epoch 49/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step - accuracy: 0.9680 - loss: 0.0745 - precision: 0.8993 - recall: 0.8910

2024-11-01 05:01:36.923173: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 05:01:36.923252: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 05:01:36.923280: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 05:01:36.923359: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 05:01:38.826324: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequen

68/68 ━━━━━━━━━━━━━━━━━━━━ 23s 335ms/step - accuracy: 0.9680 - loss: 0.0744 - precision: 0.8995 - recall: 0.8912 - val_accuracy: 0.9667 - val_loss: 0.0803 - val_precision: 0.9141 - val_recall: 0.8679
Epoch 50/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step - accuracy: 0.9675 - loss: 0.0754 - precision: 0.9004 - recall: 0.8905

2024-11-01 05:02:00.451496: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 05:02:00.451576: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 05:02:00.451743: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 05:02:00.451809: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 05:02:02.294797: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequen

68/68 ━━━━━━━━━━━━━━━━━━━━ 24s 337ms/step - accuracy: 0.9676 - loss: 0.0753 - precision: 0.9005 - recall: 0.8906 - val_accuracy: 0.9668 - val_loss: 0.0794 - val_precision: 0.9114 - val_recall: 0.8721


In [8]:
with strategy.scope():
    train_model(
        train_dataset = train_dataset, 
        val_dataset = val_dataset, 
        model = AttentionUNet3D().build_model(), 
        optimizer = 'adam',
        loss = 'binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall'],
        epochs = 50,
        batch_size = 16,
        filename="sinusoids_attentionunet3d.keras",
        model_name="sinusoids_attentionunet3d"
    )

Epoch 1/50
67/68 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step - accuracy: 0.9010 - loss: 0.2341 - precision: 0.6978 - recall: 0.7194

2024-11-01 05:03:07.341885: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 05:03:07.342171: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 05:03:07.342267: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 05:03:07.342291: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step - accuracy: 0.9014 - loss: 0.2332 - precision: 0.6990 - recall: 0.7199

2024-11-01 05:03:12.627355: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 05:03:12.627448: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 05:03:12.627515: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 05:03:12.627533: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


68/68 ━━━━━━━━━━━━━━━━━━━━ 72s 562ms/step - accuracy: 0.9018 - loss: 0.2323 - precision: 0.7001 - recall: 0.7205 - val_accuracy: 0.9385 - val_loss: 0.1434 - val_precision: 0.7713 - val_recall: 0.8610
Epoch 2/50
67/68 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step - accuracy: 0.9421 - loss: 0.1357 - precision: 0.8355 - recall: 0.7835

...

68/68 ━━━━━━━━━━━━━━━━━━━━ 30s 429ms/step - accuracy: 0.9748 - loss: 0.0590 - precision: 0.9211 - recall: 0.9184 - val_accuracy: 0.9668 - val_loss: 0.0786 - val_precision: 0.8658 - val_recall: 0.9317
Epoch 50/50
67/68 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step - accuracy: 0.9748 - loss: 0.0587 - precision: 0.9208 - recall: 0.9183

2024-11-01 05:29:38.024545: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 05:29:38.024642: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 05:29:38.024827: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 05:29:38.024929: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step - accuracy: 0.9748 - loss: 0.0587 - precision: 0.9208 - recall: 0.9183

2024-11-01 05:29:40.759110: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 05:29:40.759215: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 05:29:40.759248: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 05:29:40.759313: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


68/68 ━━━━━━━━━━━━━━━━━━━━ 30s 436ms/step - accuracy: 0.9748 - loss: 0.0587 - precision: 0.9209 - recall: 0.9182 - val_accuracy: 0.9700 - val_loss: 0.0723 - val_precision: 0.9022 - val_recall: 0.9059


# Sinusoids Augmented Dataset

In [3]:
train_dir = 'data/Sinusoids/training_data_augm/'

train_dataset, val_dataset = create_tf_datasets(
    train_dir, 
    percent_val=0.2, 
    patch_shape=(64, 64, 64), 
    patch_step=64,
    random_state = 42
)

Processing img1 image and mask
Processing img1_10_SNR image and mask
Processing img1_15_SNR image and mask
Processing img1_1_SNR image and mask
Processing img1_2_SNR image and mask
Processing img1_3_SNR image and mask
Processing img1_4_SNR image and mask
Processing img1_5_SNR image and mask
Processing img2 image and mask
Processing img2_10_SNR image and mask
Processing img2_15_SNR image and mask
Processing img2_1_SNR image and mask
Processing img2_2_SNR image and mask
Processing img2_3_SNR image and mask
Processing img2_4_SNR image and mask
Processing img2_5_SNR image and mask


In [4]:
with strategy.scope():
    train_model(
        train_dataset = train_dataset, 
        val_dataset = val_dataset, 
        model = UNet3D().build_model(), 
        optimizer = 'adam', 
        loss = 'binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall'],
        epochs = 50,
        batch_size = 16, 
        filename="sinusoids_augmented_unet3d.keras",
        model_name="sinusoids_augmented_unet3d"
    )

Epoch 1/50


2024-11-01 12:23:48.459555: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2024-11-01 12:23:48.466815: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


541/542 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step - accuracy: 0.8701 - loss: 0.5093 - precision: 0.6120 - recall: 0.3176

2024-11-01 12:26:13.675897: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 12:26:13.676185: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 12:26:13.676207: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 12:26:13.676295: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


542/542 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step - accuracy: 0.8702 - loss: 0.5088 - precision: 0.6124 - recall: 0.3181

...

542/542 ━━━━━━━━━━━━━━━━━━━━ 153s 282ms/step - accuracy: 0.9029 - loss: 0.2613 - precision: 0.4485 - recall: 0.4356 - val_accuracy: 0.9642 - val_loss: 0.0843 - val_precision: 0.9312 - val_recall: 0.8329
Epoch 49/50
542/542 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step - accuracy: 0.9038 - loss: 0.2587 - precision: 0.4489 - recall: 0.4389

2024-11-01 14:32:10.671616: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 14:32:10.671978: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 14:32:10.672035: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 14:32:10.672064: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 14:32:22.171796: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequen

542/542 ━━━━━━━━━━━━━━━━━━━━ 154s 283ms/step - accuracy: 0.9039 - loss: 0.2583 - precision: 0.4497 - recall: 0.4397 - val_accuracy: 0.9650 - val_loss: 0.0825 - val_precision: 0.8809 - val_recall: 0.8978
Epoch 50/50
542/542 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step - accuracy: 0.9049 - loss: 0.2568 - precision: 0.4497 - recall: 0.4396

2024-11-01 14:34:44.514461: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 14:34:44.514845: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 14:34:44.515092: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 14:34:44.515190: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 14:34:55.472192: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequen

542/542 ━━━━━━━━━━━━━━━━━━━━ 161s 296ms/step - accuracy: 0.9050 - loss: 0.2565 - precision: 0.4505 - recall: 0.4404 - val_accuracy: 0.9660 - val_loss: 0.0799 - val_precision: 0.9046 - val_recall: 0.8755


In [5]:
with strategy.scope():
    train_model(
        train_dataset = train_dataset, 
        val_dataset = val_dataset, 
        model = AttentionUNet3D().build_model(), 
        optimizer = 'adam',
        loss = 'binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall'],
        epochs = 50,
        batch_size = 16,
        filename="sinusoids_augmented_attentionunet3d.keras",
        model_name="sinusoids_augmented_attentionunet3d"
    )

Epoch 1/50
541/542 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step - accuracy: 0.9464 - loss: 0.1327 - precision: 0.8380 - recall: 0.8231

2024-11-01 14:39:17.331324: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 14:39:17.331685: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 14:39:17.331735: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 14:39:17.331768: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


542/542 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step - accuracy: 0.9464 - loss: 0.1327 - precision: 0.8380 - recall: 0.8231

2024-11-01 14:39:42.335148: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 14:39:42.335236: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 14:39:42.335425: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 14:39:42.335471: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


542/542 ━━━━━━━━━━━━━━━━━━━━ 254s 404ms/step - accuracy: 0.9464 - loss: 0.1326 - precision: 0.8381 - recall: 0.8232 - val_accuracy: 0.9636 - val_loss: 0.0859 - val_precision: 0.9422 - val_recall: 0.8175
Epoch 2/50
542/542 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step - accuracy: 0.9696 - loss: 0.0716 - precision: 0.9094 - recall: 0.8964

...

542/542 ━━━━━━━━━━━━━━━━━━━━ 201s 371ms/step - accuracy: 0.9834 - loss: 0.0386 - precision: 0.9490 - recall: 0.9453 - val_accuracy: 0.9807 - val_loss: 0.0462 - val_precision: 0.9351 - val_recall: 0.9421
Epoch 30/50
542/542 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step - accuracy: 0.9837 - loss: 0.0379 - precision: 0.9504 - recall: 0.9466

2024-11-01 16:18:44.259215: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 16:18:44.259460: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 16:18:44.259517: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 16:18:44.259548: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 16:19:00.495896: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequen

542/542 ━━━━━━━━━━━━━━━━━━━━ 202s 373ms/step - accuracy: 0.9837 - loss: 0.0379 - precision: 0.9504 - recall: 0.9466 - val_accuracy: 0.9809 - val_loss: 0.0455 - val_precision: 0.9482 - val_recall: 0.9288
Epoch 31/50
542/542 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step - accuracy: 0.9840 - loss: 0.0372 - precision: 0.9511 - recall: 0.9472

2024-11-01 16:22:06.847332: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 16:22:06.849899: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2024-11-01 16:22:06.849947: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 16:22:06.849982: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]
2024-11-01 16:22:22.423694: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequen

542/542 ━━━━━━━━━━━━━━━━━━━━ 202s 372ms/step - accuracy: 0.9840 - loss: 0.0372 - precision: 0.9511 - recall: 0.9473 - val_accuracy: 0.9808 - val_loss: 0.0461 - val_precision: 0.9462 - val_recall: 0.9301
